<h1 style="color:red" align="center"> Projet d'introduction au Machine Learning <br/> Out-Of-Domain PoS Tagging <br/> Présentation des résultats algorithmique</h1>

In [1]:
#chargement des corpus en mémoire

import dataAnalysis as da
import matplotlib.pyplot as plt
import numpy as np
da.init()

Initialization...
	Preparing corpus fr.foot.test.json...
	Done
	Preparing corpus fr.gsd.test.json...
	Done
	Preparing corpus fr.sequoia.train.json...
	Done
	Preparing corpus fr.ftb.test.json...
	Done
	Preparing corpus fr.ftb.dev.json...
	Done
	Preparing corpus fr.spoken.test.json...
	Done
	Preparing corpus fr.pud.train.json...
	Done
	Preparing corpus fr.spoken.train.json...
	Done
	Preparing corpus fr.natdis.test.json...
	Done
	Preparing corpus fr.partut.dev.json...
	Done
	Preparing corpus fr.sequoia.test.json...
	Done
	Preparing corpus fr.gsd.dev.json...
	Done
	Preparing corpus fr.sequoia.dev.json...
	Done
	Preparing corpus fr.gsd.train.json...
	Done
	Preparing corpus fr.pud.test.json...
	Done
	Preparing corpus fr.ftb.train.json...
	Done
	Preparing corpus fr.partut.test.json...
	Done
	Preparing corpus fr.partut.train.json...
	Done
	Preparing corpus fr.spoken.dev.json...
	Done
Done!


# Statistique des corpus

In [ ]:
#itère sur chacun des corpus
def iterateCorpus(f, printing = True):
    for nameCorpus, corpus in da.listeCorpus.items():
        if printing:
            print("corpus : ", nameCorpus)
        for typeDS, dataset in corpus.getDataset().items():
            if printing:
                print("ensemble ", typeDS)
            f(dataset, nameCorpus, typeDS)

# met à jour les données statistiques
def updateStat(dataset, nm, tds):
    dataset.updateStat()

# affiche les statistiques basique


def printBasicStat(dataset, nm, tds):
    #baseStat.append((nm + " " + tds, len(dataset.data), dataset.nbWord, dataset.nbUniqueWords))
    print("   nombre de phrase : ", len(dataset.data))
    print("   nombre de mot : ", dataset.nbWord)
    print("   nombre de mot unique : ", dataset.nbUniqueWords)


    

# affiche les 10 mots / labels les plus fréquents de chaque corpus
def printMostFrequent(dataset, nm, tds):
    print()
    print("10 mots les plus fréquents : ")
    print(dataset.mostFrequentWord)
    print()
    print()
    print("labels les plus fréquents : ")
    print(dataset.mostFrequentLabel)
    print()
    
# affiche les 10 mots les plus ambigu de chaque corpus
def printMostAmbiguousWord(dataset, nm, tds):
    ambiguousDict, ambiguousWord = dataset.ambiguousWord()
    
    def sortSecond(val):
        return len(val[1])
    
    ambiguousWord.sort(key = sortSecond, reverse = True)
    
    print()
    print("5 mots les plus ambigu : ", ambiguousWord[:5])
    print()
    
iterateCorpus(updateStat, False)

In [ ]:
#baseStat = []

iterateCorpus(printBasicStat)

from tabulate import tabulate

#print(tabulate(baseStat, tablefmt="latex", floatfmt=".2f"))

In [ ]:
iterateCorpus(printMostFrequent)

In [ ]:
#mot les plus ambigu de chaque corpus

iterateCorpus(printMostAmbiguousWord)

## Out Of Vocabulary Word

In [ ]:
# Calcul des Out of Vocabulary Words
oovLatex = []


footTest = da.listeCorpus['foot'].testDataSet
natdisTest = da.listeCorpus['natdis'].testDataSet


for nameCorpus, corpus in da.listeCorpus.items():
    if corpus.trainExist:
        oovResult = corpus.computeCorpusOOV()
        oovResultFoot = corpus.computeOOV(footTest)
        oovResultnatdis = corpus.computeOOV(natdisTest)
        print(nameCorpus, " : ")
        print("     Pourcentage de l'oov entre train et test : ", oovResult[0])
        print("     Pourcentage de l'oov entre train et dev : ", oovResult[1])
        print("     Pourcentage de l'oov entre train et footTest : ", oovResultFoot)
        print("     Pourcentage de l'oov entre train et natdisTest : ", oovResultnatdis)
        latexRows = (nameCorpus, oovResult[0], oovResult[1], oovResultFoot, oovResultnatdis)
        oovLatex.append(latexRows)

print(tabulate(oovLatex, tablefmt="latex"))

## Divergence de Kullback-Leibler

In [ ]:
# Calcul des divergence de KullBack-Leibler

latexDKL = []

for nameCorpus, corpus in da.listeCorpus.items():
    if corpus.trainExist:
        print(nameCorpus + " : ")
        DKLResult = corpus.computeCorpusKLDivergence()
        DKLFootresult = corpus.computeKLDivergence(footTest)
        DKLnatdisResult = corpus.computeKLDivergence(natdisTest)
        print("     Dkl(test||train) = ", DKLResult)
        print("     Dkl(footTest||train) = ", DKLFootresult)
        print("     Dkl(natdisTest||train) = ", DKLnatdisResult)
        latexDKL.append( (nameCorpus, DKLResult, DKLFootresult, DKLnatdisResult) )
        
        
print(tabulate(latexDKL, tablefmt="latex"))

# Perplexity

In [ ]:
latexPerplexity = []

print("                         train,                   test,              dev")
for nameCorpus, corpus in da.listeCorpus.items():
    print(nameCorpus, " : ")
    pp_res = corpus.computePerplexityCorpus()
    print("     perplexity : ", pp_res)
    latexRow = []
    latexRow.append(nameCorpus)
    if type(pp_res) == tuple:
        for i in pp_res:
            latexRow.append(i)
    else:
        latexRow.append(pp_res)
    latexPerplexity.append( latexRow )
    
print(tabulate(latexPerplexity, tablefmt="latex"))

# Training Model
## Decision Tree Classifier

### Validation croisée detection des meilleurs hyper-paramètre

In [3]:
import modelAnalysis as ma
import DecisionTreeClassifier as dtree
import features as ft

In [ ]:
# temps de calcul de cette cellule très long
for namedataset, _ in da.listeCorpus.items():
    if namedataset not in ["gsd", "ftb"]:
        ma.analyzeDecisionTree(namedataset, verbose=True)

trainSet or testSet for corpus foot not available
Beginning of the model analysis between  sequoia  train and  sequoia  test
split_criterion :  gini  / gen_test :  median
      training time :  0.16129732131958008
      Max depth :  1 score on train :  33.55093242226425   Actual score on test :  33.13141683778234  Highscore !!
      training time :  0.33096814155578613
      Max depth :  2 score on train :  35.96079149126553   Actual score on test :  35.59548254620123  Highscore !!
      training time :  0.45343494415283203
      Max depth :  3 score on train :  41.57769507656641   Actual score on test :  41.28336755646817  Highscore !!
      training time :  0.6045036315917969
      Max depth :  4 score on train :  47.46303865942692   Actual score on test :  47.002053388090346  Highscore !!
      training time :  0.7209985256195068
      Max depth :  5 score on train :  51.76417952941655   Actual score on test :  51.04722792607803  Highscore !!
      training time :  1.103799343109130

      training time :  0.47067928314208984
      Max depth :  3 score on train :  41.57769507656641   Actual score on test :  41.28336755646817  Highscore !!
      training time :  0.6067869663238525
      Max depth :  4 score on train :  46.40351412360442   Actual score on test :  45.52361396303902  Highscore !!
      training time :  0.7256114482879639
      Max depth :  5 score on train :  51.45303316860878   Actual score on test :  50.74948665297742  Highscore !!
      training time :  0.9308128356933594
      Max depth :  6 score on train :  56.34189494234641   Actual score on test :  55.780287474332646  Highscore !!
      training time :  1.2340123653411865
      Max depth :  7 score on train :  62.82106033799037   Actual score on test :  62.15605749486654  Highscore !!
      training time :  1.5823156833648682
      Max depth :  8 score on train :  68.44609846867183   Actual score on test :  67.8952772073922  Highscore !!
      training time :  2.070200204849243
      Max depth 

/home/fusroman/Documents/Informatique_et_Programmation/Intelligence Artificielle/M1_Intro_machine_learning/Projet/src/modelAnalysis.py:238: RuntimeWarning: invalid value encountered in true_divide
  cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


score on devDataSet sequoia is 89.37679967091732 with best param (17, 'entropy', 'median')
decisionTreeAnalysis on sequoia finished in 1547.208726644516 secondes
results graph can be shown in ../results/decisionTree/ location
Beginning of the model analysis between  spoken  train and  spoken  test
split_criterion :  gini  / gen_test :  median
      training time :  0.04868793487548828
      Max depth :  1 score on train :  19.094435527019797   Actual score on test :  18.76123876123876  Highscore !!
      training time :  0.09711360931396484
      Max depth :  2 score on train :  24.271000535045477   Actual score on test :  23.156843156843156  Highscore !!
      training time :  0.1629340648651123
      Max depth :  3 score on train :  32.38362760834671   Actual score on test :  30.0  Highscore !!
      training time :  0.22063875198364258
      Max depth :  4 score on train :  37.887907972177636   Actual score on test :  34.985014985014985  Highscore !!
      training time :  0.3217749

In [ ]:
# temps de calcul de cette cellule très long en raison des deux plus gros ensemble
for namedataset, _ in da.listeCorpus.items():
    if namedataset in ["gsd", "ftb"]:
        ma.analyzeDecisionTree(namedataset, verbose=True)

### Analyse du changement de contexte avec les meilleurs hyper-paramètres

In [ ]:
footTest = da.listeCorpus["foot"].testDataSet
natdisTest = da.listeCorpus["natdis"].testDataSet

Xfoot, Yfoot = ft.buildFeature(footTest)
Xnatdis, Ynatdis = ft.buildFeature(natdisTest)

In [ ]:
corpus = ["partut", "pud", "sequoia", "spoken"]
bestParam = [(15, "entropy", "mean"), (16, "entropy", "mean"), (17, "entropy", "median"), (15, "entropy", "mean")]

for nameDataset, (max_depth, split_criterion, gen_test) in zip(corpus, bestParam):
    train = da.listeCorpus[nameDataset].trainDataSet
    Xtrain, Ytrain = ft.buildFeature(train)

    
    decisionTree = dtree.DecisionTreeClassifier(Xtrain, Ytrain, 
                                                max_depth=max_depth, 
                                                split_criterion=split_criterion, 
                                                gen_test=gen_test)
    tree = decisionTree.fit()
    
    scoreFoot, cm_foot = decisionTree.modelScore(tree, Xfoot, Yfoot)
    scoreNatdis, cm_natdis = decisionTree.modelScore(tree, Xnatdis, Ynatdis)
    
    cm_foot_fig = ma.showConfusionMatrix(
            cm_foot, decisionTree.classLabel.keys(), title="confusion matrix between " + nameDataset + " train and foot test")
    
    cm_natdis_fig = ma.showConfusionMatrix(
            cm_foot, decisionTree.classLabel.keys(), title="confusion matrix between " + nameDataset + " train and natdis test")
    
    cm_foot_fig.savefig("../results/decisionTree/confusion_matrix_foot_" + nameDataset)
    cm_natdis_fig.savefig("../results/decisionTree/confusion_matrix_natdis_" + nameDataset)

## Naive Bayes Classifier

In [ ]:
import NaiveBayesClassifier as naivebc

naiveBC = naivebc.NaiveBayesClassifier(Xtrain, Ytrain)

In [ ]:
naiveBC.fit()

In [ ]:
score, cm = naiveBC.modelScore(Xtest, Ytest)

In [ ]:
print(score)

In [ ]:
ma.showConfusionMatrix(cm, decisionTree.classLabel.keys())

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
Xtrain, Ytrain = ft.buildFeature(trainSet, lambda x:x)
Xtest, Ytest = ft.buildFeature(testSet, lambda x:x)


clf = MultinomialNB()
clf.fit(trainSet, testSet)

In [ ]:
clf.score(Xtest, Ytest)